In [ ]:
# Part 5 코드 올려놓기 (퍼셉트론)
# 런타임 -> 런타임 유형 변경 -> 하드웨어 가속 -> GPU -> 저장
# 을 반드시 하고 나서 연결 눌러주세요
# 안 그러면 수백배 느림!!!

In [ ]:
# 딥러닝 프레임워크
# 텐서플로우(케라스)랑 파이토치

In [ ]:
import torch # 라이브러리 이름은 PyTorch
from torch import nn # Neural Network (신경망)
from torch.utils.data import DataLoader # 데이터로더 기능: (다운로드, 학습/검증 스플릿, 텐서로 변환)
# 코드가 좀 Dense하다고 느낄 수 있음
# = 짤막한 코드가 수행하는 기능이 여럿이다
from torchvision import datasets # 예를 들면 CIFAR-10, COCO, MNIST, Fashion-MNIST, ...
# 비전 관련 라이브러리: torchvision
# 오디오: torchaudio
# 텍스트: torchtext
# 강화학습: torch-rl
from torchvision.transforms import ToTensor, Compose, Normalize # 배열을 텐서로 바꾸기
# torchvision에서 구현되어있는 ToTensor는 이미지를 받아서 텐서로 바꿈
# 1. 자료형이 바뀜 -> PIL Image 객체를 텐서로 바꿈
# 2. 축 재배열 (transpose 연산) (Height, Width, Channel) -> (Channel, Height, Width)

In [ ]:
transform = Compose([ # 이미지데이터에 적용할 일련의 변환들을 리스트 인자로 받음
    ToTensor(),
    Normalize((0.0,), (1.0,)) # 정규화 (각 축에 대해서)
])

In [ ]:

# 판다스 실습할 때 root_dir = "drive/MyDrive/..."
train_data = datasets.FashionMNIST(
    root="data", # data란 이름의 폴더에 저장함
    train=True, # training 용도인지 boolean으로 주면 됨
    download=True, # 다운로드하겠다
    transform=transform, # ToTensor 함수로 받은 이미지를 변환해달라
)

test_data = datasets.FashionMNIST(
    root="data", # data란 이름의 폴더에 저장함
    train=False, # training 용도인지 boolean으로 주면 됨
    download=True, # 다운로드하겠다
    transform=transform, # ToTensor 함수로 받은 이미지를 변환해달라
)
# gzip 프로그램의 압축 결과물 .gz
# gzip GNU Zip; GNU -> GNU is Not Unix! # recursive abbreviation
# GNU/Unix

In [ ]:
batch_size = 64 # mini-batch 경사 하강 -> 프로그래밍할 때는 이 구분을 그대로 따르지 않을 수도 있음
# batch_size 64로 해놓고 최적화기법을 SGD로 쓴다든지 할 수 있음

# batch_size = 1이면 SGD
# batch_size = num_data면 batch 경사 하강
train_dataloader = DataLoader(train_data, batch_size=batch_size)
# hubo = Robot()
# 데이터로더 인스턴스 만듦
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# 데이터로더는 순회 가능함

for X, y in test_dataloader:
  print(X.shape, type(X)) # [64, 1, 28, 28] -> N, C, H, W -> N: batch_size; C: channel 갯수; H: height; W: width
  print(y.shape, type(y)) # [64] -> batch_size 개의 라벨만 나옴
  break

torch.Size([64, 1, 28, 28]) <class 'torch.Tensor'>
torch.Size([64]) <class 'torch.Tensor'>


In [ ]:
if torch.cuda.is_available():
  device = "cuda"
else:
  "cpu"
# CUDA는 하드웨어 가속 라이브러리
# GPU에서 텐서 연산이 빨리 돌게끔 도와줌

In [ ]:
# 클래스 선언
# nn.Module로부터 상속함
# nn은 torch 하위 신경망 라이브러리
# Module? 신경망 만들으라고 torch에서 구현해놓은 base class
# Module의 메서드? 예) forward  입력층에서 텐서를 출력층까지로 보내는 연산

# 모델 만들기
# FCNN랑 CNN 비교해보자
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__() # 피상속 클래스의 생성자 호출 -> 기초과정 내용 중에 Animal() my_dog = Dog(name="Nami")

    self.flatten = nn.Flatten() # nn신경망 라이브러리에 정의되어있는 Flatten 클래스 인스턴스 만들기
    # 뭘 납작하게? --> 입력 데이터
    # keras.layers에도 Flatten이 있음
    # keras.models.Sequential도 있음
    # 인자로 층들을 넣어주면 해당 층들을 순차적으로 가지는 (부분)신경망을 만들어줌
    self.fcnn = nn.Sequential(
        # 예를 들면 폭 512로 만들자
        # 28 * 28 = 784
        # PCA? 차원 축소 -> explained_variance_ratio
        # 28*28은 고정해주세요 (데이터 특성이 이러니까)
        nn.Linear(28*28, 512), # torch에서는 Linear라고 불림; keras.layers에서는 Dense라고 불림; 우리 슬라이드에서는 Fully Connected
        # 인자로 들어간 두 값 -> 층의 폭. 즉, 28*28=784개의 폭(노드 갯수)를 가진 층과 다음 층(폭이 512인)을 fully connect 하겠다
        nn.ReLU(),
        nn.Dropout(0.2),

        # 직전 층 폭과 일치하는 512
        nn.Linear(512, 128), # 예를 들면 폭 128로 만들자
        nn.ReLU(),
        nn.Dropout(0.3),

        # 직전 층 폭과 일치하는 128
        nn.Linear(128, 10) # 10? 클래스 갯수
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.fcnn(x) # FC층에서 나온 그 값들 자체
    # logits 중 가장 큰 값을 가지는 그 위치를 답으로 내겠다는 뜻
    # 3개 이상 클래스 분류할 때 softmax를 씀
    # 개념적으로는 이진 분류 logistic 활성화 함수의 n개 클래스 확장 버전
    # return nn.softmax(logits) 라고 해서 클래스 별 확률을 바로 NeuralNetwork 모델 선언 단에서
    # 계산하기도 함
    return logits

In [ ]:
class NeuralNetworkStudent1(nn.Module): # 0.8553 기록함 epochs=15, lr=1e-3
  def __init__(self):
    super().__init__() # 피상속 클래스의 생성자 호출 -> 기초과정 내용 중에 Animal() my_dog = Dog(name="Nami")

    self.flatten = nn.Flatten() # nn신경망 라이브러리에 정의되어있는 Flatten 클래스 인스턴스 만들기
    # 뭘 납작하게? --> 입력 데이터
    # keras.layers에도 Flatten이 있음
    # keras.models.Sequential도 있음
    # 인자로 층들을 넣어주면 해당 층들을 순차적으로 가지는 (부분)신경망을 만들어줌
    self.fcnn = nn.Sequential(
        # 예를 들면 폭 512로 만들자
        # 28 * 28 = 784
        # PCA? 차원 축소 -> explained_variance_ratio
        # 28*28은 고정해주세요 (데이터 특성이 이러니까)
        nn.Linear(28*28, 512), # torch에서는 Linear라고 불림; keras.layers에서는 Dense라고 불림; 우리 슬라이드에서는 Fully Connected
        # 인자로 들어간 두 값 -> 층의 폭. 즉, 28*28=784개의 폭(노드 갯수)를 가진 층과 다음 층(폭이 512인)을 fully connect 하겠다
        nn.ReLU(),
        nn.BatchNorm1d(512),

        # 직전 층 폭과 일치하는 512
        nn.Linear(512, 128), # 예를 들면 폭 128로 만들자
        nn.ReLU(),


        # 직전 층 폭과 일치하는 128
        nn.Linear(128, 10) # 10? 클래스 갯수
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.fcnn(x) # FC층에서 나온 그 값들 자체
    # logits 중 가장 큰 값을 가지는 그 위치를 답으로 내겠다는 뜻
    # 3개 이상 클래스 분류할 때 softmax를 씀
    # 개념적으로는 이진 분류 logistic 활성화 함수의 n개 클래스 확장 버전
    # return nn.softmax(logits) 라고 해서 클래스 별 확률을 바로 NeuralNetwork 모델 선언 단에서
    # 계산하기도 함
    return logits

In [ ]:
class NeuralNetworkStudent2(nn.Module):
  def __init__(self):
    super().__init__() # 피상속 클래스의 생성자 호출 -> 기초과정 내용 중에 Animal() my_dog = Dog(name="Nami")

    self.flatten = nn.Flatten() # nn신경망 라이브러리에 정의되어있는 Flatten 클래스 인스턴스 만들기
    # 뭘 납작하게? --> 입력 데이터
    # keras.layers에도 Flatten이 있음
    # keras.models.Sequential도 있음
    # 인자로 층들을 넣어주면 해당 층들을 순차적으로 가지는 (부분)신경망을 만들어줌
    self.fcnn = nn.Sequential(
        # 예를 들면 폭 512로 만들자
        # 28 * 28 = 784
        # PCA? 차원 축소 -> explained_variance_ratio
        # 28*28은 고정해주세요 (데이터 특성이 이러니까)
        nn.Linear(28*28, 512), # torch에서는 Linear라고 불림; keras.layers에서는 Dense라고 불림; 우리 슬라이드에서는 Fully Connected
        # 인자로 들어간 두 값 -> 층의 폭. 즉, 28*28=784개의 폭(노드 갯수)를 가진 층과 다음 층(폭이 512인)을 fully connect 하겠다
        nn.ReLU(),
        nn.Dropout(0.2),

        # 직전 층 폭과 일치하는 512
        nn.Linear(512, 128), # 예를 들면 폭 128로 만들자
        nn.ReLU(),

        nn.BatchNorm1d(128),
        # 직전 층 폭과 일치하는 128
        nn.Linear(128, 10) # 10? 클래스 갯수
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.fcnn(x) # FC층에서 나온 그 값들 자체
    # logits 중 가장 큰 값을 가지는 그 위치를 답으로 내겠다는 뜻
    # 3개 이상 클래스 분류할 때 softmax를 씀
    # 개념적으로는 이진 분류 logistic 활성화 함수의 n개 클래스 확장 버전
    # return nn.softmax(logits) 라고 해서 클래스 별 확률을 바로 NeuralNetwork 모델 선언 단에서
    # 계산하기도 함
    return logits

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__() # 피상속 클래스의 생성자 호출 -> 기초과정 내용 중에 Animal() my_dog = Dog(name="Nami")
    self.cnn = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1),
        # 특성 맵 사이즈: 24
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.MaxPool2d(kernel_size=2),
        # 특성 맵 사이즈: 12
        nn.Conv2d(32, out_channels=64, kernel_size=5),
        # 특성 맵 사이즈: 8
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        # 특성 맵 사이즈: 4
        nn.Flatten(),
        # 텐서 사이즈: 64 * 4 * 4 = 1024
        # 직전 층 폭과 일치하는 512
        nn.Linear(1024, 128), # 예를 들면 폭 128로 만들자
        nn.ReLU(),
        nn.BatchNorm1d(128),
        # 직전 층 폭과 일치하는 128
        nn.Linear(128, 10) # 10? 클래스 갯수
    )

  def forward(self, x):
    logits = self.cnn(x) # FC층에서 나온 그 값들 자체
    # logits 중 가장 큰 값을 가지는 그 위치를 답으로 내겠다는 뜻
    # 3개 이상 클래스 분류할 때 softmax를 씀
    # 개념적으로는 이진 분류 logistic 활성화 함수의 n개 클래스 확장 버전
    # return nn.softmax(logits) 라고 해서 클래스 별 확률을 바로 NeuralNetwork 모델 선언 단에서
    # 계산하기도 함
    return logits

In [ ]:
class CNNDemo(nn.Module):
  def __init__(self):
    super().__init__() # 피상속 클래스의 생성자 호출 -> 기초과정 내용 중에 Animal() my_dog = Dog(name="Nami")
    self.cnn = nn.Sequential(
        nn.Conv2d(1, 32, 5),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(32, 64, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Flatten(),
        nn.Linear(1024, 128),
        nn.ReLU(),
        nn.BatchNorm1d(128),
        nn.Linear(128, 10)
    )

  def forward(self, x):
    logits = self.cnn(x) # FC층에서 나온 그 값들 자체
    # logits 중 가장 큰 값을 가지는 그 위치를 답으로 내겠다는 뜻
    # 3개 이상 클래스 분류할 때 softmax를 씀
    # 개념적으로는 이진 분류 logistic 활성화 함수의 n개 클래스 확장 버전
    # return nn.softmax(logits) 라고 해서 클래스 별 확률을 바로 NeuralNetwork 모델 선언 단에서
    # 계산하기도 함
    return logits

In [ ]:
1152/32

36.0

In [ ]:
# model = NeuralNetwork().to(device)
# model = NeuralNetworkStudent1().to(device)
# model = NeuralNetworkStudent2().to(device)
model = CNNDemo().to(device)

# 신경망 인스턴스 만들고 -> CUDA로 "보냄" (GPU 상에서 작동할 수 있도록)
print(model)

CNNDemo(
  (cnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=1024, out_features=128, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
######
# 모델 튜닝 해보기
# epochs
# optimizer의 lr 인자
# 신경망의 층 구성 (층 갯수, 각 층 폭)
# Dropout 어디에 넣을지, dropout rate
# BatchNorm1D 어디에 넣을지
# CNN에서는: out_channels, stride, kernel_size, pooling

In [ ]:
# 학습시키기 위해서는
# 손실/비용 함수
# 최적화기법
# 정해줘야 함
loss = nn.CrossEntropyLoss() # log loss
# SparseCrossEntropyLoss를 쓰면 분류 결과값이 y가 one-hot 인코딩과 비교될 때
# sparse 텅 빈 -> 출력 텐서에서 한 값만 1이라
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 7 * 10^(-4)
# SGD: stochastic gradient descent이지만 슬라이드 기준으로는 mini-batch gradient descent (batch_size=64)
# 첫번째 인자: 뭘 최적화하는지 -> 모델 파라미터들
# 두번째 인자: Learning Rate 잘 모르겠다면 0.001 ~ 0.0001 사이 어디서부턴가 시작하자

In [ ]:
# 약간은 기본적인 기능 low-level인 것부터 직접 짜보고
# 나중에 편의기능 써보기
def train(dataloader, model_, loss_, optimizer_):
  # 데이터셋 사이즈
  size = len(dataloader.dataset)
  model_.train() # 학습 모드로 두겠다 -> 파라미터 값이 갱신되도록 하겠다
  # dataloader는 iterable 자료형이니까
  # enumerate로 순번(index)이랑 그 위치의 데이터 (X, y) 튜플을 동시에 접근 가능
  # dataloader 만들었을 때 이미 batch_size 64 정해줬기 때문에
  # dataloader에서 꺼내지는 batch, (X, y) 도 batch_size 개만큼 꺼내짐
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) # cuda에 있는 모델에게 "보내자"
    yhat = model_(X) # nn.Module 클래스 정의에 따라 "forward"라 불리는 함수를 직접 호출하지 않아도
    # 모델 이름 자체를 함수처럼 쓸 수 있음 -> 그러면 forward가 알아서 호출됨
    loss_value = loss_(yhat, y) # 비용 함수 (J) 계산함

    # 여기서부터 역전파 (backprop)
    loss_value.backward() # 미분값 계산하고, chain rule 적용해서 이전 층의 모든 노드들에게 미분값 전달
    # 자동 미분
    optimizer_.step() # 최적화 기법에 따라 한 "걸음" 파라미터 보정
    optimizer_.zero_grad() # 파라미터 보정 한번 했으니까 (이번 batch에 대해서) 이제 초기화

    if batch % 500 == 0:
      current_loss, current = loss_value.item(), (batch+1) * len(X)
      # PyTorch 기준 텐서에서 값만 꺼내서 보고 싶을 때 item() 함수 호출하면 됨
      # Pandas DataFrame에서 값(들)만 꺼내서 보고 싶을 때 df.values, df.to_numpy()
      # current는 지금 몇번째 데이터를 보고 있는지
      print(current_loss, " at data index ", current)

In [ ]:
def test(dataloader, model_, loss_): # 테스트 단이니까 optimizer가 작동할 일이 없음
  # 데이터셋 사이즈
  size = len(dataloader.dataset)
  num_batches = len(dataloader) # 데이터로더에서 배치 단위로 꺼내왔었음
  # 즉, 데이터로더는 배치 갯수만큼의 길이를 가짐
  model_.eval() # 검증 모드로 두겠다 -> 파라미터 값이 갱신 안 되도록 하겠다
  test_loss, correct = 0, 0 # 검증 단계 비용, 맞힌 문제 갯수
  # dataloader는 iterable 자료형이니까
  # enumerate로 순번(index)이랑 그 위치의 데이터 (X, y) 튜플을 동시에 접근 가능
  # dataloader 만들었을 때 이미 batch_size 64 정해줬기 때문에
  # dataloader에서 꺼내지는 batch, (X, y) 도 batch_size 개만큼 꺼내짐

  with torch.no_grad(): # 기울기 계산을 안 하겠다
    for X, y in dataloader: # 배치 단위로 꺼내기
      X, y = X.to(device), y.to(device)
      yhat = model_(X)
      test_loss += loss_(yhat, y).item() # 누적 비용 계산을 위해 += 연산
      # 값만 추적 (텐서 형태 불필요) --> .item() 호출
      correct += (yhat.argmax(1) == y).type(torch.float).sum().item()
      # 몇 문제 맞혔는지 누계 추적 변수 correct
      # 텐서의 메서드 중 argmax는 인자 갯수만큼 상위 값들의 위치를 반환
      # y.shape 기억해보세요 -> (60000,) 즉 y는 스칼라 0~9
      # yhat은 폭이 10인 텐서임 (신경망 마지막 층 구조 참고)
      # 그 텐서에서 가장 값이 큰 1개의 위치(index)가 y와 일치하는지 여부 (True/False)
      # 캐스팅 파이썬 기본 문법 (float(2)); pandas.DataFrame(사전)
      # 파이토치 캐스팅 문법에 따라 boolean 값을 0(False) 또는 1(True)로 바꿈
      # 0 또는 1값을 가진 길이 batch_size인 텐서가 나옴
      # 그 텐서 sum을 구하겠다는 뜻은 -> 1 갯수를 세겠다
  test_loss /= num_batches # 배치 당 평균 비용
  correct /= size
  print("정답률 ", correct, " 배치 당 평균 비용 ", test_loss)
  # with open("파일이름.txt") as f:
    # 이 scope 안에서는 f를 쓰고, scope가 끝나면 f 자동 close() 호출
  return

In [ ]:
epochs = 10 # 모든 데이터 샘플 한번 보면 epoch
for ep in range(epochs):
  print("Epoch ", ep)
  train(train_dataloader, model, loss, optimizer)
  test(test_dataloader, model, loss)

Epoch  0
2.561415910720825  at data index  64
0.8183695077896118  at data index  32064
정답률  0.7921  배치 당 평균 비용  0.7493315612434581
Epoch  1
0.6013126969337463  at data index  64
0.6357166767120361  at data index  32064
정답률  0.8254  배치 당 평균 비용  0.6045353799868541
Epoch  2
0.4685196578502655  at data index  64
0.5583020448684692  at data index  32064
정답률  0.834  배치 당 평균 비용  0.539679508869815
Epoch  3
0.3949597477912903  at data index  64
0.5390070080757141  at data index  32064
정답률  0.8528  배치 당 평균 비용  0.4836218218514874
Epoch  4
0.3450136184692383  at data index  64
0.49453479051589966  at data index  32064
정답률  0.8574  배치 당 평균 비용  0.45785979765235996
Epoch  5
0.3131885230541229  at data index  64
0.45866042375564575  at data index  32064
정답률  0.8578  배치 당 평균 비용  0.44125552807643914
Epoch  6
0.28596219420433044  at data index  64
0.4273832142353058  at data index  32064
정답률  0.8652  배치 당 평균 비용  0.41584788538088463
Epoch  7
0.2819063067436218  at data index  64
0.3981167674064636  at dat